<a href="https://colab.research.google.com/github/Speedbird45Bravo/rando_projects/blob/main/Torch_6922.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
from torch.nn import Module, Sequential, Linear, Conv2d, ReLU, MaxPool2d, CrossEntropyLoss
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torch.optim import Adam
from pytz import timezone
from datetime import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [26]:
train = MNIST("/files", train=True, download=True, transform=ToTensor())
test = MNIST("/files", train=False, download=True, transform=ToTensor())
loaders = {"train":DataLoader(train, num_workers=4, shuffle=True, batch_size=64),\
           "test":DataLoader(train, num_workers=4, shuffle=True, batch_size=64)}

In [27]:
class NeuralNetwork(Module):

  def __init__(self):
    
    super(NeuralNetwork, self).__init__()
    self.c1 = Sequential(Conv2d(1,16,5,1,2),ReLU(),MaxPool2d(2))
    self.c2 = Sequential(Conv2d(16,32,5,1,2),ReLU(),MaxPool2d(2))
    self.out = Sequential(Linear(32 * 7 * 7, 10))

  def forward(self, x):
    x = self.c1(x)
    x = self.c2(x)
    x = x.view(x.size(0),-1)
    output = self.out(x)
    return output, x

In [28]:
tz = timezone("US/Eastern")
cnn = NeuralNetwork()
print(cnn)
print(f"Network printed at {dt.now(tz=tz)}")

NeuralNetwork(
  (c1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (c2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Sequential(
    (0): Linear(in_features=1568, out_features=10, bias=True)
  )
)
Network printed at 2022-06-09 20:40:02.346602-04:00


In [29]:
epochs = 6
loss_function = CrossEntropyLoss()
rabinowitz = Adam(cnn.parameters(), lr=0.01)

In [38]:
def train(cnn, epochs, loaders):

  cnn.train()

  for epoch in range(epochs):
    
    for i, (images, labels) in enumerate(loaders["train"]):
      b_X = Variable(images)
      b_y = Variable(labels)
      output = cnn(b_X)[0]
      loss = loss_function(output, b_y)

      rabinowitz.zero_grad()

      loss.backward()

      rabinowitz.step()

      if (i + 1) % 100 == 0:
        print(f"Epoch: {epoch+1}/{epochs} | Loss: {loss.item()}")

  print(f"Model Trained at {dt.now(tz=tz)}")

In [39]:
train(cnn, epochs, loaders)

Epoch: 1/6 | Loss: 0.04065423458814621
Epoch: 1/6 | Loss: 0.062293462455272675
Epoch: 1/6 | Loss: 0.020458277314901352
Epoch: 1/6 | Loss: 0.038256216794252396
Epoch: 1/6 | Loss: 0.01141560822725296
Epoch: 1/6 | Loss: 0.007013246417045593
Epoch: 1/6 | Loss: 0.16946765780448914
Epoch: 1/6 | Loss: 0.03629615902900696
Epoch: 1/6 | Loss: 0.04748102277517319
Epoch: 2/6 | Loss: 0.07532931864261627
Epoch: 2/6 | Loss: 0.003262479789555073
Epoch: 2/6 | Loss: 0.022120216861367226
Epoch: 2/6 | Loss: 0.17363499104976654
Epoch: 2/6 | Loss: 0.10468323528766632
Epoch: 2/6 | Loss: 0.31312018632888794
Epoch: 2/6 | Loss: 0.04865209013223648
Epoch: 2/6 | Loss: 0.0065182787366211414
Epoch: 2/6 | Loss: 0.003868302796036005
Epoch: 3/6 | Loss: 0.0041288044303655624
Epoch: 3/6 | Loss: 0.000285739719402045
Epoch: 3/6 | Loss: 0.09741739183664322
Epoch: 3/6 | Loss: 0.0012608522083610296
Epoch: 3/6 | Loss: 0.0024540573358535767
Epoch: 3/6 | Loss: 0.19656941294670105
Epoch: 3/6 | Loss: 0.12336269021034241
Epoch: 3/

In [40]:
def test():

  cnn.eval()

  with torch.no_grad():

    total = 0

    correct = 0

    for images, labels in loaders["test"]:
      test_output, last_layer = cnn(images)
      y_predictions = torch.max(test_output,1)[1].data.squeeze()
      accuracy = (y_predictions == labels).sum().item() / float(labels.size(0))
      pass

  print(f"Test Accuracy: {round(accuracy * 100, 3)}%")
  print(f"Model Tested at {dt.now(tz=tz)}")

In [41]:
test()

Test Accuracy: 100.0%
Model Tested at 2022-06-09 20:51:11.615674-04:00
